In [1]:
import sys
sys.path.append('../../')

from utils import linmix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import astropy.units as u

sns.set_theme()
sns.set_style("ticks")
sns.set_context("paper")

u_flux = u.erg / u.cm**2 / u.s**1

## Line Regression Results
Here I will compute correlation coefficients using the algorithm from [Kelly (2007)](https://ui.adsabs.harvard.edu/abs/2007ApJ...665.1489K/abstract),
and Spitzer line fluxes and disks properties given by Dr. Andrea Banzatti.

The goal for now is to compare my regression and correlation coefficients to those of [Banzatti et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903..124B/abstract).

First let's load the data into memory so we can start working with it.

In [2]:
spitzer = pd.read_csv('../../Data/Spitzer_ALMA_sample.csv', sep=',', skipinitialspace=True, na_values=['#NAME?'])
spitzer.head()

,NAME,MSTAR,LSTAR,REFMSTAR,LOGLACC,LOGMACC,REFLACC,RHOLE,HOLE,RINGS,...,FLCO_NC,FLCO,ERRCO,FLH2O_3,ERRH2O_3,FLOH_3,ERROH_3,TEMP_S11,DENS_S11,DET_S11
0,04385+2550,0.50,0.62,Andrews+2018(GAIA),-1.228,-8.20,Andrews+2018(GAIA),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,AATau,0.60,0.50,Herczeg&Hillenbrand+2014,-1.430,-8.31,Salyk+2013,28.0,mm,mm,...,0.000000e+00,2.750000e-14,1.910000e-16,1.420000e-15,1.420000e-15,1.040000e-15,3.390000e-17,400.0,18.8,1.0
2,AS205N,0.87,2.14,Andrews+2018(GAIA),-0.070,-6.58,Fang+2018(GAIA),NaN,NaN,NaN,...,1.340000e-13,3.130000e-13,8.950000e-14,7.330000e-14,2.710000e-14,2.060000e-14,7.610000e-15,300.0,20.6,1.0
3,AS209,0.96,0.87,Fang+2018(GAIA),-1.120,-8.30,Fang+2018(GAIA),NaN,NaN,mm,...,7.090000e-15,1.740000e-14,4.100000e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,250.0,20.9,1.0
4,BPTau,0.54,0.40,Fang+2018(GAIA),-1.170,-8.14,Fang+2018(GAIA),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


## Setup Data Collection Procedure

We are going to perform the MCMC regression sampling procedure on multiple pairs of datasets.
The easiest way to perform the same procedure on multiple different things is to use a loop.

Before we start looping over pairs of datasets we need to setup the parameters which we will loop over.
In this case we will loop over the line fluxes (y-values), and the disk sample properties (x-values).
So we can save them into lists that we will later loop over; `sample_properties` and `line_fluxes` (`line_fluxes_err`).
We will need the distance properties too for converting to luminosities so we save it in `dist`.

Finally we make an empty table that we will be filling in using the linear regression output.
I will adopt the table structure from Table 1. in [Banzatti et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903..124B/abstract),
so that I can easily compare it to Dr. Banzatti's coefficients.

In [3]:
# Lists of important stuff for the loops

sample_properties = ['LOGLACC', 'LOGRDUST95', 'N1331']
line_fluxes = ['FLH2O_17', 'FLHCN', 'FLC2H2' ,'FLCO2']
line_fluxes_err = ['ERRH2O_17', 'ERRHCN', 'ERRC2H2', 'ERRCO2']
dist = spitzer['DIST']

# Create empty table that looks like Table 1. from Andrea 2020
cols = pd.MultiIndex.from_product([sample_properties, ['alpha','alpha_std' , 'beta', 'beta_std', 'corr', 'sigma']])
add_index = ['FLH2O_17/FLHCN', 'FLH2O_17/FLC2H2', 'FLH2O_17/FLCO2']

df = pd.DataFrame(columns=cols, index=line_fluxes + add_index)
df

LOGLACC                                    LOGRDUST95  \
                  alpha alpha_std beta beta_std corr sigma      alpha   
FLH2O_17            NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLHCN               NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLC2H2              NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLCO2               NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLH2O_17/FLHCN      NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLH2O_17/FLC2H2     NaN       NaN  NaN      NaN  NaN   NaN        NaN   
FLH2O_17/FLCO2      NaN       NaN  NaN      NaN  NaN   NaN        NaN   

                                                   N1331                 \
                alpha_std beta beta_std corr sigma alpha alpha_std beta   
FLH2O_17              NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLHCN                 NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLC2H2                NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLCO2                 NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLH2O_17/FLHCN        NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLH2O_17/FLC2H2       NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   
FLH2O_17/FLCO2        NaN  NaN      NaN  NaN   NaN   NaN       NaN  NaN   

                                     
                beta_std corr sigma  
FLH2O_17             NaN  NaN   NaN  
FLHCN                NaN  NaN   NaN  
FLC2H2               NaN  NaN   NaN  
FLCO2                NaN  NaN   NaN  
FLH2O_17/FLHCN       NaN  NaN   NaN  
FLH2O_17/FLC2H2      NaN  NaN   NaN  
FLH2O_17/FLCO2       NaN  NaN   NaN

## Luminosities

Now we are ready to loop over our parameters.
We can perform a double loop, one over the line fluxes, and a second over the disk properties.
There are several steps for the procedure right before computing and sampling the line regression coefficients.
First we ge the data we are looping over from the `spitzer` DataFrame, and then we filter out any NaNs.
Then we swap in upper limits as per instructions from Dr. Banzatti.
Finally we the delta function array, convert fluxes to luminosities, and run the MCMC sampler.

After that all is left is to collect the variables we want to save into our empty DataFrame `df` and write in the values.


In [4]:
# Parameters of current run
for flux_param, flux_param_err in zip(line_fluxes, line_fluxes_err):
    for disk_param in sample_properties:
        print(disk_param, flux_param, flux_param_err)

        # Get Data
        line_flux = spitzer[flux_param]
        line_flux_err = spitzer[flux_param_err]
        disk_property = spitzer[disk_param]

        # Filter NaNs
        filter_nans = ~(np.isnan(line_flux) | np.isnan(disk_property))
        line_flux = line_flux[filter_nans]
        line_flux_err = line_flux_err[filter_nans]
        disk_property = disk_property[filter_nans].values

        # Swap upper limits in flux measurements
        flux_upp_mask = (line_flux_err > line_flux).values
        line_flux_corr = line_flux.copy()
        line_flux_corr[flux_upp_mask] = 2 * line_flux_err[flux_upp_mask]

        # Get Delta function
        delta = (~flux_upp_mask).astype(int)

        # Calculate luminosity of line flux; using astropy units and convert to Lsun
        lum =  (4 * np.pi * line_flux_corr * dist[filter_nans]**2 * (u.pc**2 * u_flux) ).values.to(u.Lsun).value

        # Compute regression coefficients, and other stuff
        lm = linmix.LinMix(disk_property, np.log10(lum), delta=delta)
        lm.run_mcmc(maxiter=10000, silent=True)
        
        # Compute median values, standard deviations of results
        beta_median = np.median(lm.chain['beta'])
        beta_std = np.std(lm.chain['beta'])

        alpha_median = np.median(lm.chain['alpha'])
        alpha_std = np.std(lm.chain['alpha'])

        corr_median = np.median(lm.chain['corr'])
        sigma_median = np.median(np.sqrt(lm.chain['sigsqr']))
        
        # and save data into the dataframe
        row = alpha_median, alpha_std, beta_median, beta_std, corr_median, sigma_median
        df.loc[flux_param, disk_param] = row

LOGLACC FLH2O_17 ERRH2O_17
LOGRDUST95 FLH2O_17 ERRH2O_17
N1331 FLH2O_17 ERRH2O_17
LOGLACC FLHCN ERRHCN
LOGRDUST95 FLHCN ERRHCN
N1331 FLHCN ERRHCN
LOGLACC FLC2H2 ERRC2H2
LOGRDUST95 FLC2H2 ERRC2H2
N1331 FLC2H2 ERRC2H2
LOGLACC FLCO2 ERRCO2
LOGRDUST95 FLCO2 ERRCO2
N1331 FLCO2 ERRCO2


In [5]:
df

LOGLACC                                                    \
                    alpha alpha_std      beta  beta_std      corr     sigma   
FLH2O_17        -4.017958  0.094171  0.523111  0.075264  0.701628  0.391858   
FLHCN           -4.119079  0.084482  0.444971  0.067818    0.6821  0.352269   
FLC2H2          -4.364108  0.092353  0.480814  0.073265  0.679157  0.383988   
FLCO2           -4.663327  0.100142  0.367877  0.079749  0.545434  0.416059   
FLH2O_17/FLHCN        NaN       NaN       NaN       NaN       NaN       NaN   
FLH2O_17/FLC2H2       NaN       NaN       NaN       NaN       NaN       NaN   
FLH2O_17/FLCO2        NaN       NaN       NaN       NaN       NaN       NaN   

                LOGRDUST95                                                    \
                     alpha alpha_std      beta  beta_std      corr     sigma   
FLH2O_17         -3.919319  0.335768 -0.383676  0.191752 -0.258635  0.519638   
FLHCN            -4.263308  0.307458 -0.194926  0.175665 -0.147196  0.476803   
FLC2H2           -4.848296  0.334738 -0.023196  0.190669 -0.016265  0.517748   
FLCO2            -4.925248  0.316246 -0.072942  0.179529 -0.054844  0.484596   
FLH2O_17/FLHCN         NaN       NaN       NaN       NaN       NaN       NaN   
FLH2O_17/FLC2H2        NaN       NaN       NaN       NaN       NaN       NaN   
FLH2O_17/FLCO2         NaN       NaN       NaN       NaN       NaN       NaN   

                    N1331                                                    
                    alpha alpha_std      beta  beta_std      corr     sigma  
FLH2O_17        -4.579936  0.066591 -0.161491   0.07846 -0.256115  0.513574  
FLHCN           -4.589363   0.05863 -0.175387  0.068723 -0.310821  0.451857  
FLC2H2          -4.896574  0.063122 -0.114762   0.07479 -0.194951  0.485101  
FLCO2           -5.049727  0.061217  -0.10854  0.071695 -0.187625  0.477058  
FLH2O_17/FLHCN        NaN       NaN       NaN       NaN       NaN       NaN  
FLH2O_17/FLC2H2       NaN       NaN       NaN       NaN       NaN       NaN  
FLH2O_17/FLCO2        NaN       NaN       NaN       NaN       NaN       NaN

## Normalized Fluxes 

Our DataFrame `df` is incomplete for now.
Table 1. in [Banzatti et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903..124B/abstract) includes ratios between flux lines,
so we can do that too. That way we have more "data points" from which we can compare the linear regression algorithm I use 
to that of _Banzatti et al. 2020_.

However, I am not sure how to handle the arithmetic using upper limits; I am not sure if I need to give any special treatment.
So we will wait until I talk to Dr. Banzatti about this.
__For now this is a work in progress.__



In [6]:
# Get param strings
flux_param1, flux_param2 = add_index[0].split('/')
disk_param = 'LOGLACC'

# Get Data
line_flux1 = spitzer[flux_param1]
line_flux_err1 = spitzer[line_fluxes_err[line_fluxes.index(flux_param1)]]

line_flux2 = spitzer[flux_param2]
line_flux_err2 = spitzer[line_fluxes_err[line_fluxes.index(flux_param2)]]

disk_property = spitzer[disk_param]

# Filter NaNs
filter_nans = ~(np.isnan(line_flux1) | np.isnan(line_flux2) | np.isnan(disk_property))
line_flux1 = line_flux1[filter_nans]
line_flux_err1 = line_flux_err1[filter_nans]
line_flux2 = line_flux2[filter_nans]
line_flux_err2 = line_flux_err2[filter_nans]
disk_property = disk_property[filter_nans].values

############ UPPER LIMIT AND NON-UPPER LIMIT ARITHMETIC IS WEIRD AND I DONT GET IT
############ I will need to divide upper limits in both numerator and demoninator, but is that correct?

# Swap upper limits in flux measurements
# flux_upp_mask = (line_flux_err > line_flux).values
# line_flux_corr = line_flux.copy()
# line_flux_corr[flux_upp_mask] = 2 * line_flux_err[flux_upp_mask]


In [7]:
## I WILL EXPORT THE DATA HERE
df.to_csv('line_regression_results.csv', index_label='Name')